Name:
Student Id:

### Task 1: Naive Formula + Uniform Weighting

In [ ]:
## Write your code here


In [ ]:
## This part is for discussion



### Task2: Naive Formula + Uniform Weighting + Non-Maximum Suppression

In [ ]:
## Write your code here


In [ ]:
## This part is for discussion



### Task3: Taylor's Approximation + Uniform Weighting + Non-Maximum Suppression

In [ ]:
## Write your code here


In [ ]:
## This part is for discussion



### Task 4: Smaller Eigenvalue as corner score + Uniform Weighting + Non-Maximum Suppression

In [ ]:
## Write your code here


In [ ]:
## This part is for discussion



### Task 5: R function as corner score + Uniform Weighting + Non-Maximum Suppression

In [ ]:
## Write your code here


In [ ]:
## This part is for discussion



### Task 6: R function with fast windowing based on fitering + Uniform Weighting + Non-Maximum Suppression

In [ ]:
## Write your code here


In [ ]:
## This part is for discussion



### Task 7: R function with fast windowing based on fitering + Gaussian Weighting + Non-Maximum Suppression

In [ ]:
## Write your code here


In [ ]:
## This part is for discussion

